### 特征提取

#### TF-IDF

**Term frequency-inverse document frequency (TF-IDF)**是在文本挖掘中常用的特征向量化方法，用来反映一个**单词（term）**在**语料库（corpus）**中对于一个**文档（document）**的重要程度。

标记单词为t，文档为d，语料库为D

**词频（term frequency）**TF(t,d) 表示单词t在文档d中出现的次数

**文档频度（document frequency）**DF(t,D) 表示包含单词t的文档的个数

如果只使用词频（term frequency），会过于强调出现次数多的单词，这些单词可能并没有包含有用的信息（例如：a、the、of）。

如何一个单词在语料库中到处存在，则说明这个单词在特定的文档中并没有特殊的含义。

**逆文档频率（inverse document frequency）**是一个单词提供信息的量化表示：

$IDF(t, D) = \log \frac{|D| + 1}{DF(t, D) + 1}$

|D|表示语料库中文档的总数

**TF-IDF** 就是**TF**和**IDF**的乘积：

$TFIDF(t, d, D) = TF(t, d) \cdot IDF(t, D)$

#### 例子

In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
        .builder\
        .appName("IfIdfExample")\
        .getOrCreate()

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

使用 Tokenizer 做分词

In [5]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

计算 TF

In [6]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

计算 TF-IDF

In [7]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [8]:
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[6,8,13,16],[...|
|  0.0|(20,[0,2,7,13,15,...|
|  1.0|(20,[3,4,6,11,19]...|
+-----+--------------------+



In [9]:
spark.stop()